<a href="https://colab.research.google.com/github/pragyayadav/CarsAnalysis/blob/main/Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Car Recognition with Keras (TensorFlow backend) on a single node

This notebook is an attempt to use Keras (with TensorFlow in the backend) on the Spark driver node to fit a neural network on the Stanford Cars Dataset

Prerequisites:
* A GPU-enabled cluster on Databricks.
* Keras and TensorFlow installed with GPU support.

The content of this notebook was based on one [copied from the Keras project](https://github.com/fchollet/keras/blob/47350dc6078053403c59e8da3fd63ac3ae12b5ec/examples/mnist_cnn.py) under the [MIT license](https://github.com/fchollet/keras/blob/47350dc6078053403c59e8da3fd63ac3ae12b5ec/LICENSE) with slight modifications in comments. Thanks to the developers of Keras for this example!

**Note:** None of this will work unless you have also uploaded all the required notebooks to the same Databricks directory (conv_block, identity_block, res_net, resnet_utils, and scale_layer)!

### Car Recognition

This tutorial guides you through a computer vision exampl: identify images of cars with neural networks. 
We will train a Residual Convolutional Neural Network on the Stanford Cars dataset.

Note that we will not explicitly choose a backend for Keras, so it will use TensorFlow by default.

In [ ]:
import numpy as np
import sys
from keras import layers, callbacks
from keras.optimizers import SGD, Adam
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
import scipy.misc

#Clear Memory
import gc
gc.collect()

import os
cwd = os.getcwd()
print("Current working directory is " + cwd)

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
Current working directory is /databricks/driver

### Import Important Stuff
For some reason these all have to be executed in a separate cells, without any comments (don't ask me why)

In [ ]:
%run "/Users/evan@datainsights.de/resnets_utils"

In [ ]:
%run "/Users/evan@datainsights.de/scale_layer"

In [ ]:
%run "/Users/evan@datainsights.de/identity_block"

In [ ]:
%run "/Users/evan@datainsights.de/conv_block"

In [ ]:
%run "/Users/evan@datainsights.de/res_net"

### Set Some Stuff

In [ ]:
img_width = 224
img_height = 224
classes = 196
batch_size = 16
epochs = 30 #Be sure to set this based on how long you want to run for!
patience = 50 #For Callbacks
weights_freq = epochs #How often to save weights (right now set to only save on the last epoch, as saving the weights takes 30+mins on databricks)
verbose = 1
num_train_samples = 6549
num_valid_samples = 1695 #Cross validation: num_train_samples + num_valid_samples = # of train images
mode = 'sgd' #adam or sgd

### Initialize the Mode

In [ ]:
#Build the model:
model = ResNet(input_shape = (img_width, img_height, 3), classes = classes)
if (mode == 'sgd'): optimizer = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True) #SGD seems to work better
if (mode == 'adam'): optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False) #lr is default 1e-3
try:
      model.load_weights("/dbfs/evan/weights.best.hdf5")
      print("Reading in previous weight file...")
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
except:
      print("Weight file not found. A new one will be created.")
      model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Reading in previous weight file...

### Import Data and Set-up Training Parameters (Callbacks, etc.)

We use the [Stanford Cars](https://ai.stanford.edu/~jkrause/cars/car_dataset.html) dataset, which contains 16,185 images of cars, divided into 196 different models. Keras provides a handy function for loading this data.

In [ ]:
# Load Dataset (note that test data is loaded separately lower down - this helps with memory)
X_train, Y_train_orig, classes = load_train_dataset()

# It's not necessary to normalize X_train because of the Scale step in the neural network (after BatchNorm)

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 196).T

# If cross validation is desired, then take some of the training data for this purpose
if (num_valid_samples > 0):
	X_valid = X_train[num_train_samples::,:,:,:]
	Y_valid = Y_train[num_train_samples::]
	X_train = X_train[0:num_train_samples,:,:,:]
	Y_train = Y_train[0:num_train_samples]
	

print ("number of training examples = " + str(X_train.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_valid shape: " + str(X_valid.shape))
print ("Y_valid shape: " + str(Y_valid.shape))

# Data Augmentation and Cross Validation
train_data_gen = image.ImageDataGenerator(rotation_range=20., width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True)
train_data_gen.fit(X_train)
train_generator = train_data_gen.flow(X_train, Y_train, batch_size=batch_size)
valid_data_gen = image.ImageDataGenerator()
valid_generator = valid_data_gen.flow(X_valid, Y_valid, batch_size=batch_size)

# Callbacks
checkpoint = callbacks.ModelCheckpoint("/dbfs/evan/weights.best.hdf5", monitor='val_acc', verbose=verbose, save_best_only=False, save_weights_only=False, mode='auto', period=weights_freq)
csv_logger = callbacks.CSVLogger('/dbfs/evan/training.log', append=True)
early_stop = callbacks.EarlyStopping('val_acc', patience=patience)
reduce_lr = callbacks.ReduceLROnPlateau('val_acc', factor=0.1, patience=int(patience/4), verbose=1)
callbacks=[csv_logger,checkpoint,early_stop,reduce_lr]

number of training examples = 6549
X_train shape: (6549, 224, 224, 3)
Y_train shape: (6549, 196)
X_valid shape: (1595, 224, 224, 3)
Y_valid shape: (1595, 196)

### Train the Model

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=num_train_samples/batch_size, validation_data=valid_generator, validation_steps=num_valid_samples/batch_size, epochs=epochs, callbacks=callbacks, verbose=verbose)

WARNING:tensorflow:From /databricks/python/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30

 1/409 [..............................] - ETA: 2:48:32 - loss: 0.0056 - acc: 1.0000
 2/409 [..............................] - ETA: 1:24:57 - loss: 0.0068 - acc: 1.0000
 3/409 [..............................] - ETA: 57:02 - loss: 0.0060 - acc: 1.0000 
 4/409 [..............................] - ETA: 43:05 - loss: 0.0065 - acc: 1.0000
 5/409 [..............................] - ETA: 34:43 - loss: 0.0065 - acc: 1.0000
 6/409 [..............................] - ETA: 29:08 - loss: 0.0067 - acc: 1.0000
 7/409 [..............................] - ETA: 25:09 - loss: 0.0120 - acc: 1.0000
 8/409 [..............................] - ETA: 22:09 - loss: 0.0113 - acc: 1.0000
 9/409 [..............................] - ETA: 19:49 - loss: 0.0108 - acc: 1.0000
 10/409 [..............................] - ETA: 17:58 - loss: 0.0103 - acc: 1.0000
 11/409 [..............................] - ETA: 16:26 - loss: 0.0107 - acc: 1.0000
 12/409 [..............................] - ETA: 15:09 - loss: 0.0100 - acc: 1.0000
 13/409 [..............................] - ETA: 14:04 - loss: 0.0107 - acc: 1.0000
 14/409 [>.............................] - ETA: 13:08 - loss: 0.0102 - acc: 1.0000
 15/409 [>.............................] - ETA: 12:20 - loss: 0.0103 - acc: 1.0000
 16/409 [>.............................] - ETA: 11:38 - loss: 0.0102 - acc: 1.0000
 17/409 [>.............................] - ETA: 11:00 - loss: 0.0097 - acc: 1.0000
 18/409 [>.............................] - ETA: 10:27 - loss: 0.0094 - acc: 1.0000
 19/409 [>.............................] - ETA: 9:57 - loss: 0.0091 - acc: 1.0000 
 20/409 [>.............................] - ETA: 9:30 - loss: 0.0090 - acc: 1.0000
 21/409 [>.............................] - ETA: 9:06 - loss: 0.0092 - acc: 1.0000
 22/409 [>.............................] - ETA: 8:43 - loss: 0.0091 - acc: 1.0000
 23/409 [>.............................] - ETA: 8:23 - loss: 0.0088 - acc: 1.0000
 24/409 [>.............................] - ETA: 8:05 - loss: 0.0090 - acc: 1.0000
 25/409 [>.............................] - ETA: 7:48 - loss: 0.0088 - acc: 1.0000
 26/409 [>.............................] - ETA: 7:32 - loss: 0.0089 - acc: 1.0000
 27/409 [>.............................] - ETA: 7:17 - loss: 0.0086 - acc: 1.0000
 28/409 [=>............................] - ETA: 7:04 - loss: 0.0086 - acc: 1.0000
 29/409 [=>............................] - ETA: 6:52 - loss: 0.0086 - acc: 1.0000
 30/409 [=>............................] - ETA: 6:40 - loss: 0.0086 - acc: 1.0000
 31/409 [=>............................] - ETA: 6:29 - loss: 0.0090 - acc: 1.0000
 32/409 [=>............................] - ETA: 6:18 - loss: 0.0091 - acc: 1.0000
 33/409 [=>............................] - ETA: 6:09 - loss: 0.0091 - acc: 1.0000
 34/409 [=>............................] - ETA: 5:59 - loss: 0.0089 - acc: 1.0000
 35/409 [=>............................] - ETA: 5:51 - loss: 0.0089 - acc: 1.0000
 36/409 [=>............................] - ETA: 5:43 - loss: 0.0089 - acc: 1.0000
 37/409 [=>............................] - ETA: 5:35 - loss: 0.0088 - acc: 1.0000
 38/409 [=>............................] - ETA: 5:27 - loss: 0.0093 - acc: 1.0000
 39/409 [=>............................] - ETA: 5:20 - loss: 0.0091 - acc: 1.0000
 40/409 [=>............................] - ETA: 5:13 - loss: 0.0091 - acc: 1.0000
 41/409 [==>...........................] - ETA: 5:08 - loss: 0.0093 - acc: 1.0000
 42/409 [==>...........................] - ETA: 5:02 - loss: 0.0101 - acc: 1.0000
 43/409 [==>...........................] - ETA: 4:57 - loss: 0.0099 - acc: 1.0000
 44/409 [==>...........................] - ETA: 4:51 - loss: 0.0100 - acc: 1.0000
 45/409 [==>...........................] - ETA: 4:46 - loss: 0.0100 - acc: 1.0000
 46/409 [==>..........

### Test the Model

In [ ]:
#Tidy up memory:
X_train_orig = Y_train_orig = X_train = Y_train = None

#Imoprt stuff
X_test, Y_test_orig, classes = load_test_dataset()
Y_test = convert_to_one_hot(Y_test_orig, 196).T

print ("number of test examples = " + str(X_test.shape[0]))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

number of test examples = 8041
X_test shape: (8041, 224, 224, 3)
Y_test shape: (8041, 196)

 32/8041 [..............................] - ETA: 1:36
 64/8041 [..............................] - ETA: 1:00
 96/8041 [..............................] - ETA: 48s 
 128/8041 [..............................] - ETA: 41s
 160/8041 [..............................] - ETA: 38s
 192/8041 [..............................] - ETA: 35s
 224/8041 [..............................] - ETA: 33s
 256/8041 [..............................] - ETA: 32s
 288/8041 [>.............................] - ETA: 30s
 320/8041 [>.............................] - ETA: 30s
 352/8041 [>.............................] - ETA: 29s
 384/8041 [>.............................] - ETA: 28s
 416/8041 [>.............................] - ETA: 28s
 448/8041 [>.............................] - ETA: 27s
 480/8041 [>.............................] - ETA: 27s
 512/8041 [>.............................] - ETA: 26s
 544/8041 [=>............................] - ETA: 26s
 576/8041 [=>............................] - ETA: 25s
 608/8041 [=>............................] - ETA: 25s
 640/8041 [=>............................] - ETA: 25s
 672/8041 [=>............................] - ETA: 25s
 704/8041 [=>............................] - ETA: 24s
 736/8041 [=>............................] - ETA: 24s
 768/8041 [=>............................] - ETA: 24s
 800/8041 [=>............................] - ETA: 24s
 832/8041 [==>...........................] - ETA: 23s
 864/8041 [==>...........................] - ETA: 23s
 896/8041 [==>...........................] - ETA: 23s
 928/8041 [==>...........................] - ETA: 23s
 960/8041 [==>...........................] - ETA: 23s
 992/8041 [==>...........................] - ETA: 23s
1024/8041 [==>...........................] - ETA: 22s
1056/8041 [==>...........................] - ETA: 22s
1088/8041 [===>..........................] - ETA: 22s
1120/8041 [===>..........................] - ETA: 22s
1152/8041 [===>..........................] - ETA: 22s
1184/8041 [===>..........................] - ETA: 22s
1216/8041 [===>..........................] - ETA: 21s
1248/8041 [===>..........................] - ETA: 21s
1280/8041 [===>..........................] - ETA: 21s
1312/8041 [===>..........................] - ETA: 21s
1344/8041 [====>.........................] - ETA: 21s
1376/8041 [====>.........................] - ETA: 21s
1408/8041 [====>.........................] - ETA: 21s
1440/8041 [====>.........................] - ETA: 20s
1472/8041 [====>.........................] - ETA: 20s
1504/8041 [====>.........................] - ETA: 20s
1536/8041 [====>.........................] - ETA: 20s
1568/8041 [====>.........................] - ETA: 20s
1600/8041 [====>.........................] - ETA: 20s
1632/8041 [=====>........................] - ETA: 20s
1664/8041 [=====>........................] - ETA: 20s
1696/8041 [=====>........................] - ETA: 19s
1728/8041 [=====>........................] - ETA: 19s
1760/8041 [=====>........................] - ETA: 19s
1792/8041 [=====>........................] - ETA: 19s
1824/8041 [=====>........................] - ETA: 19s
1856/8041 [=====>........................] - ETA: 19s
1888/8041 [======>.......................] - ETA: 19s
1920/8041 [======>.......................] - ETA: 19s
1952/8041 [======>.......................] - ETA: 18s
1984/8041 [======>.......................] - ETA: 18s
2016/8041 [======>.......................] - ETA: 18s
2048/8041 [======>.......................] - ETA: 18s
2080/8041 [======>.......................] - ETA: 18s
2112/8041 [======>.......................] - ETA: 18s
2144/8041 [======>.......................] - ETA: 18s
2176/8041 [=======>......................] - ETA: 18s
2208/8041 [=======>......................] - ETA: 18s
2240/8041 [=======>......................] - ETA: 17s
2272/8041 [=======>......................] - ETA: 17s
2304/8041 [=======>......................] - ETA: 17s
2336/8041 [=======>.